# FedLib: Heterogeneous Federated Learning using Dynamic Model Pruning and Adaptive Gradient

## Importing supportive libaries
This notebook shows a demo on PyTorch back-end model impelementation.

In the very begining, we import the supporting libraries.

In [1]:
import torch
import numpy as np
import copy

from fedlib.ve import FEDDFEnv as Simulator
from fedlib.lib import Server, Client
from fedlib.networks import resnet20
from fedlib.lib.sampler import random_sampler
from fedlib.lib.algo import feddp
from fedlib.datasets import partition_data, get_dataloader,get_client_dataloader
from fedlib.utils import get_logger

## Define arguments
Here we define arguments. To show an intuitive example, we show the demo store all the parameters in a dictionary in the following code block.
We also provide APIs for you create your arguments in a `*.yaml` file.

In [2]:
args = {}
args["n_clients"] = 100
args["device"] = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args['sample_fn'] = random_sampler
args['trainer'] = feddp(logger=get_logger())
args['communicator'] = None
args["test_dataset"] = None
args["partition"] = "noniid-labeldir"
args["dataset"] = "cifar10"
args["datadir"] = "./data"
args["beta"] = 0.5
args["batch_size"] = 64
args["global_model"] = resnet20()
args["lr"] = 0.01
args["optimizer"] = "SGD"
args["lr_scheduler"] = "ExponentialLR"

Load test dataset for server, and passing it as an argument

In [3]:
X_train, y_train, X_test, y_test, net_dataidx_map, traindata_cls_counts = partition_data(
    args["dataset"], args["datadir"], args['partition'], args['n_clients'], beta=args['beta'])
n_classes = len(np.unique(y_train))
train_dl_global, test_dl_global, train_ds_global, test_ds_global = get_dataloader(args["dataset"],
                                                                                    args["datadir"],
                                                                                      args["batch_size"],
                                                                                      32)
args["test_dataset"] = test_dl_global

Files already downloaded and verified
Files already downloaded and verified


INFO:root:Data statistics: {0: {0: 20, 1: 24, 2: 40, 3: 69, 4: 98, 5: 10, 7: 108, 8: 68, 9: 39}, 1: {0: 80, 1: 111, 2: 95, 3: 3, 4: 121, 5: 17, 6: 1, 8: 185}, 2: {0: 3, 1: 84, 2: 14, 3: 69, 4: 4, 5: 51, 6: 18, 7: 1, 8: 145, 9: 28}, 3: {0: 169, 1: 47, 2: 38, 3: 2, 4: 42, 5: 2, 6: 1, 7: 50, 9: 8}, 4: {0: 45, 1: 32, 2: 10, 3: 4, 4: 12, 5: 109, 6: 9, 7: 6, 8: 19, 9: 12}, 5: {0: 138, 1: 4, 2: 15, 3: 25, 4: 15, 5: 112, 6: 1, 7: 8, 8: 49, 9: 33}, 6: {0: 5, 1: 1, 2: 84, 3: 31, 4: 61, 5: 2, 6: 6, 7: 19, 8: 74, 9: 13}, 7: {0: 83, 1: 10, 2: 8, 3: 5, 4: 48, 5: 81, 6: 9, 7: 18, 8: 6, 9: 22}, 8: {0: 1, 1: 1, 2: 41, 3: 39, 4: 74, 5: 18, 6: 83, 7: 1, 8: 63, 9: 38}, 9: {0: 142, 1: 23, 2: 19, 3: 91, 4: 2, 5: 178, 6: 27, 7: 3, 8: 86}, 10: {0: 73, 1: 11, 2: 30, 3: 7, 5: 39, 6: 3, 7: 2, 8: 17, 9: 2}, 11: {0: 8, 2: 93, 3: 37, 4: 12, 5: 19, 6: 7, 7: 128, 8: 13}, 12: {0: 4, 1: 9, 2: 70, 3: 8, 4: 1, 5: 27, 7: 90, 8: 10, 9: 5}, 13: {0: 3, 1: 1, 2: 112, 3: 129, 4: 155, 5: 15, 6: 25, 7: 344}, 14: {0: 131, 1: 45, 

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Create server and clients objects
Here we use the arguments we defined before, and create server and clients.

In [4]:
server = Server(**args)
clients = {}

data_loaders, global_test_dl, test_loaders = get_client_dataloader(args["dataset"], args["datadir"], args['batch_size'], 32, net_dataidx_map)


Files already downloaded and verified
Files already downloaded and verified


INFO:root:Client ID:1,	Local Train Data Size:50000
INFO:root:Client ID:2,	Local Train Data Size:50000
INFO:root:Client ID:3,	Local Train Data Size:50000
INFO:root:Client ID:4,	Local Train Data Size:50000
INFO:root:Client ID:5,	Local Train Data Size:50000
INFO:root:Client ID:6,	Local Train Data Size:50000
INFO:root:Client ID:7,	Local Train Data Size:50000
INFO:root:Client ID:8,	Local Train Data Size:50000
INFO:root:Client ID:9,	Local Train Data Size:50000
INFO:root:Client ID:10,	Local Train Data Size:50000
INFO:root:Client ID:11,	Local Train Data Size:50000
INFO:root:Client ID:12,	Local Train Data Size:50000
INFO:root:Client ID:13,	Local Train Data Size:50000
INFO:root:Client ID:14,	Local Train Data Size:50000
INFO:root:Client ID:15,	Local Train Data Size:50000
INFO:root:Client ID:16,	Local Train Data Size:50000
INFO:root:Client ID:17,	Local Train Data Size:50000
INFO:root:Client ID:18,	Local Train Data Size:50000
INFO:root:Client ID:19,	Local Train Data Size:50000
INFO:root:Client ID:2

In [5]:


for id in range(args["n_clients"]):
    # dataidxs = net_dataidx_map[id]
    args["id"] = id
    # args["trainloader"], _, _, _ = get_dataloader(args["dataset"], args["datadir"], args['batch_size'], 32, dataidxs)
    args["trainloader"] = data_loaders[id]
    args["testloader"] =global_test_dl
    args["model"] = copy.deepcopy(args["global_model"])
    args['criterion'] = torch.nn.CrossEntropyLoss()
    clients[id] = Client(**args)

    


## Create simulator

Simulator simulates the virtual federated learning environments, and run server and clients on single device.

In [6]:
simulator = Simulator(server=server, clients=clients, communication_rounds=10,n_clients= 100,sample_rate=.1)

## Run simulator
User API Simulator.run

In [7]:
simulator.run(local_epochs=1,pruning_threshold=1e-3)

INFO:root:*******starting Rounds 1 Optimization******
INFO:root:Participate Clients: [ 1 60 88  4 69 38 30 33 26 89]
INFO:root:Optimize the 1-th Clients
INFO:root:Layer: conv1, Sparsity: 0.46%
INFO:root:Layer: layer1.0.conv1, Sparsity: 0.65%
INFO:root:Layer: layer1.0.conv2, Sparsity: 0.65%
INFO:root:Layer: layer1.1.conv1, Sparsity: 0.74%
INFO:root:Layer: layer1.1.conv2, Sparsity: 0.61%
INFO:root:Layer: layer1.2.conv1, Sparsity: 0.65%
INFO:root:Layer: layer1.2.conv2, Sparsity: 0.74%
INFO:root:Layer: layer2.0.conv1, Sparsity: 0.61%
INFO:root:Layer: layer2.0.conv2, Sparsity: 1.00%
INFO:root:Layer: layer2.1.conv1, Sparsity: 1.04%
INFO:root:Layer: layer2.1.conv2, Sparsity: 1.00%
INFO:root:Layer: layer2.2.conv1, Sparsity: 0.97%
INFO:root:Layer: layer2.2.conv2, Sparsity: 1.01%
INFO:root:Layer: layer3.0.conv1, Sparsity: 0.87%
INFO:root:Layer: layer3.0.conv2, Sparsity: 1.34%
INFO:root:Layer: layer3.1.conv1, Sparsity: 1.44%
INFO:root:Layer: layer3.1.conv2, Sparsity: 1.30%
INFO:root:Layer: layer3